In [1]:
import datetime,string,os
import numpy as np
from collections import defaultdict

testset = False
if testset:
	folder = "./test/"
else:
	folder = "./train/"

# get start and enddate of all users
user_dates = {}
with open("./data/user_dates.csv","r") as fi:
    fi.readline()
    for lines in fi:
        li = lines.strip().split(",")
        user_dates[li[0]] = {'firstdate' : datetime.datetime.strptime(li[1],"%Y-%m-%d").date(),'lastdate' : datetime.datetime.strptime(li[2],"%Y-%m-%d").date() } # transform string to date

# get categories we are interested in
categories = set([])
for e, line in enumerate( open("./offers.csv","r") ):
	if e > 0:
		li = line.strip().split(",")
		categories.add(li[1])
categories = list(categories) #優惠券可使用的種類共20種

with open( os.path.join(folder, "category_repeat_buy_probability.csv"),"w") as of:
    of.write("category_id,cat_repeat_buy_prob_30d,cat_repeat_buy_prob_60d,cat_repeat_buy_prob_90d\n")

    for c in categories:
        print("calculating repeat-probability for category "+c)  #計算可能重複購買的商品種類(有優惠券的商品種類)

        repeaters_30d = []
        repeaters_60d = []
        repeaters_90d = []

        with open("./data/reduced.csv","r")as fi:  
            fi.readline()
            # set standard variables
            bought_pre_6m = defaultdict(bool)
            firstbuy_date = {}
            repeatbuy_dates = defaultdict(list)
            lastid = 0

            for e,lines in enumerate(fi):
                li = lines.strip().split(",")
                if not lastid == li[0] and e > 0: 
                    # get all stuff
                    for p in bought_pre_6m.keys(): # p is product number {productid : True}
                        if p in firstbuy_date:  # 此商品在第一次購買時
                            if (user_dates[user]['lastdate']-firstbuy_date[p]).days > 30: # 交易歷史紀錄中最後一次消費日期與此商品第一次購買日期間隔 
                                repeat_within_30d = 0   # 大於三十天,則設定三十日重複購買為零
                                # check if it's repeated within thirty days
                                for d in repeatbuy_dates[p]: # 重複購買與第一次購買此商品的間隔天數小於三十天
                                    if (d-firstbuy_date[p]).days < 30: 
                                        repeat_within_30d = 1
                                repeaters_30d.append(repeat_within_30d) # 三十天重複購買次數加一
                            if (user_dates[user]['lastdate']-firstbuy_date[p]).days > 60:
                                repeat_within_60d = 0
                                # check if it's repeated within thirty days
                                for d in repeatbuy_dates[p]:
                                    if (d-firstbuy_date[p]).days < 60:
                                        repeat_within_60d = 1
                                repeaters_60d.append(repeat_within_60d)
                            if (user_dates[user]['lastdate']-firstbuy_date[p]).days > 90:
                                repeat_within_90d = 0
                                # check if it's repeated within thirty days
                                for d in repeatbuy_dates[p]:
                                    if (d-firstbuy_date[p]).days < 90:
                                        repeat_within_90d = 1
                                repeaters_90d.append(repeat_within_90d)

                    # reset variables ??
                    bought_pre_6m = defaultdict(bool)
                    firstbuy_date = {}
                    repeatbuy_dates = defaultdict(list)		
                user = li[0]
                category = li[3]
                product = " ".join(li[3:6])
                date = datetime.datetime.strptime(li[6],"%Y-%m-%d").date()
                if category == c and float(li[10]) >= 0 and user in user_dates:  # 商品種類是優惠券的商品種類  
                    if (date - user_dates[user]['firstdate']).days < 30*6:  # 過去半年有此商品種類購買紀錄    
                        # register if user bought the product in first half of year
                        bought_pre_6m[product] = True
                    elif bought_pre_6m[product] == False:   # 若過去半年未有購買紀錄
                        # candidate for calculation
                        if not product in firstbuy_date:    # 此商品id未在第一次購買串列中則設定此日期為第一次購買日 
                            firstbuy_date[product] = date
                        else:
                            # append to dates
                            repeatbuy_dates[product].append(date)  #有購買過則將此日期加入重複購買日
                lastid = li[0]
                if e % 1000000 == 0 and e > 0:
                    print(e)

        # write out probabilities
        rep_30 = float(np.sum(repeaters_30d))/float(len(repeaters_30d))
        rep_60 = float(np.sum(repeaters_60d))/float(len(repeaters_60d))
        rep_90 = float(np.sum(repeaters_90d))/float(len(repeaters_90d))
        print("done with "+c)
        print("number items for 30d : "+str(len(repeaters_30d))) # 分母
        print("rep30 : "+str(rep_30))
        print("rep60 : "+str(rep_60))
        print("rep90 : "+str(rep_90))
        of.write(c+","+str(rep_30)+","+str(rep_60)+","+str(rep_90)+"\n")

calculating repeat-probability for category 5616
done with 5616
number items for 30d : 124
rep30 : 0.13709677419354838
rep60 : 0.16216216216216217
rep90 : 0.20689655172413793
calculating repeat-probability for category 799
done with 799
number items for 30d : 2
rep30 : 0.0
rep60 : 0.0
rep90 : 0.0
calculating repeat-probability for category 3203
done with 3203
number items for 30d : 46
rep30 : 0.06521739130434782
rep60 : 0.10810810810810811
rep90 : 0.14285714285714285
calculating repeat-probability for category 6202
done with 6202
number items for 30d : 32
rep30 : 0.21875
rep60 : 0.2916666666666667
rep90 : 0.36363636363636365
calculating repeat-probability for category 5824
done with 5824
number items for 30d : 104
rep30 : 0.0673076923076923
rep60 : 0.11827956989247312
rep90 : 0.2345679012345679
calculating repeat-probability for category 3509
done with 3509
number items for 30d : 53
rep30 : 0.11320754716981132
rep60 : 0.17391304347826086
rep90 : 0.225
calculating repeat-probability for